In [7]:
#설치 확인
import torch
print(torch.__version__)

2.7.0+cpu


In [1]:
!pip install pytorch-tabnet

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [4]:
#데이터 준비
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

train_df = pd.read_csv('TRAIN_DATA.csv', encoding='euc-kr')
test_df = pd.read_csv('TEST_DATA.csv', encoding='euc-kr')

#데이터 확인
print(train_df.head())
print(test_df.head())

   DATA_YM  AREA_ID AREA_NM  DIST_CD DIST_NM  TOTAL_BIDG  FAC_NEIGH_1  \
0   202303     9626  중앙로역_4    27110      중구       538.0        266.0   
1   202303    10350    황리단길    47130     경주시       765.0         72.0   
2   202212     9547    농소1동    31200      북구       343.0         74.0   
3   202312     9995  심석고등학교    41360    남양주시       129.0          4.0   
4   202301     9248     고덕역    11740     강동구        34.0          2.0   

   FAC_NEIGH_2  FAC_CULT_MTG  FAC_RELG  FAC_RETAIL  FAC_MEDI  FAC_YOSE  \
0         88.0           6.0       3.0         2.0       3.0       0.0   
1        174.0           1.0       0.0         2.0       0.0       1.0   
2        161.0           1.0       3.0         0.0       2.0       6.0   
3         14.0           0.0       1.0         0.0       0.0       0.0   
4         14.0           0.0       0.0         1.0       0.0       1.0   

   FAC_TRAIN  FAC_SPORT  FAC_STAY  FAC_LEISURE  TOTAL_GAS  CMRC_GAS  \
0        0.0        0.0      16.0          3.

In [11]:
print(test_df.columns)

Index(['DATA_YM', 'AREA_ID', 'AREA_NM', 'DIST_CD', 'DIST_NM', 'TOTAL_BIDG',
       'FAC_NEIGH_1', 'FAC_NEIGH_2', 'FAC_CULT_MTG', 'FAC_RELG', 'FAC_RETAIL',
       'FAC_MEDI', 'FAC_YOSE', 'FAC_TRAIN', 'FAC_SPORT', 'FAC_STAY',
       'FAC_LEISURE', 'TOTAL_GAS', 'CMRC_GAS'],
      dtype='object')


In [5]:

# AREA_ID와 AREA_NM 컬럼 리스트로 추출
train_areas = train_df[['AREA_ID', 'AREA_NM']].drop_duplicates()
test_areas = test_df[['AREA_ID', 'AREA_NM']].drop_duplicates()

# AREA_ID와 AREA_NM을 기준으로 겹치지 않는 값 찾기
train_ids = set(zip(train_areas['AREA_ID'], train_areas['AREA_NM']))
test_ids = set(zip(test_areas['AREA_ID'], test_areas['AREA_NM']))

# 겹치지 않는 값 찾기
train_unique = train_ids - test_ids
test_unique = test_ids - train_ids

# DIST_CD와 DIST_NM 컬럼 리스트로 추출
train_dist = train_df[['DIST_CD', 'DIST_NM']].drop_duplicates()
test_dist = test_df[['DIST_CD', 'DIST_NM']].drop_duplicates()

# DIST_CD와 DIST_NM 기준으로 겹치지 않는 값 찾기
train_cds = set(zip(train_dist['DIST_CD'], train_dist['DIST_NM']))
test_cds = set(zip(test_dist['DIST_CD'], test_dist['DIST_NM']))

# 겹치지 않는 값 찾기
train_unique2 = train_cds - test_cds
test_unique2 = test_cds - train_cds

# 결과 출력
print("Train 데이터에만 있는 AREA_ID와 AREA_NM:")
for area in train_unique:
    print(area)

print("\nTest 데이터에만 있는 AREA_ID와 AREA_NM:")
for area in test_unique:
    print(area)

print("\nTrain 데이터에만 있는 DIST_CD와 DIST_NM:")
for area in train_unique2:
    print(area)

print("\nTest 데이터에만 있는 DIST_CD와 DIST_NM:")
for area in test_unique2:
    print(area)   



Train 데이터에만 있는 AREA_ID와 AREA_NM:
(9888, '수원시청_1')
(9946, '유천아파트앞')

Test 데이터에만 있는 AREA_ID와 AREA_NM:

Train 데이터에만 있는 DIST_CD와 DIST_NM:
(47720, '군위군')
(42720, '홍천군')
(42730, '횡성군')

Test 데이터에만 있는 DIST_CD와 DIST_NM:


In [6]:
# Train 데이터에만 있는 값들
train_remove_ids = [(9888, '수원시청_1'), (9946, '유천아파트앞')]
train_remove_dist = [(42720, '홍천군'), (42730, '횡성군'), (47720, '군위군')]

# Train 데이터에서 삭제
for area_id, area_name in train_remove_ids:
    train_df = train_df[~((train_df['AREA_ID'] == area_id) & (train_df['AREA_NM'] == area_name))]

for dist_cd, dist_name in train_remove_dist:
    train_df = train_df[~((train_df['DIST_CD'] == dist_cd) & (train_df['DIST_NM'] == dist_name))]

# 확인하기!!
print("===== 출력되는게 없으면 성공 =====")
# AREA_ID와 AREA_NM 컬럼 리스트로 추출
train_areas = train_df[['AREA_ID', 'AREA_NM']].drop_duplicates()
test_areas = test_df[['AREA_ID', 'AREA_NM']].drop_duplicates()

# AREA_ID와 AREA_NM을 기준으로 겹치지 않는 값 찾기
train_ids = set(zip(train_areas['AREA_ID'], train_areas['AREA_NM']))
test_ids = set(zip(test_areas['AREA_ID'], test_areas['AREA_NM']))

# 겹치지 않는 값 찾기
train_unique = train_ids - test_ids
test_unique = test_ids - train_ids

# DIST_CD와 DIST_NM 컬럼 리스트로 추출
train_dist = train_df[['DIST_CD', 'DIST_NM']].drop_duplicates()
test_dist = test_df[['DIST_CD', 'DIST_NM']].drop_duplicates()

# DIST_CD와 DIST_NM 기준으로 겹치지 않는 값 찾기
train_cds = set(zip(train_dist['DIST_CD'], train_dist['DIST_NM']))
test_cds = set(zip(test_dist['DIST_CD'], test_dist['DIST_NM']))

# 겹치지 않는 값 찾기
train_unique2 = train_cds - test_cds
test_unique2 = test_cds - train_cds

# 결과 출력
print("\nTrain 데이터에만 있는 AREA_ID와 AREA_NM:")
for area in train_unique:
    print(area)

print("\nTest 데이터에만 있는 AREA_ID와 AREA_NM:")
for area in test_unique:
    print(area)

print("\nTrain 데이터에만 있는 DIST_CD와 DIST_NM:")
for area in train_unique2:
    print(area)

print("\nTest 데이터에만 있는 DIST_CD와 DIST_NM:")
for area in test_unique2:
    print(area)   



===== 출력되는게 없으면 성공 =====

Train 데이터에만 있는 AREA_ID와 AREA_NM:

Test 데이터에만 있는 AREA_ID와 AREA_NM:

Train 데이터에만 있는 DIST_CD와 DIST_NM:

Test 데이터에만 있는 DIST_CD와 DIST_NM:


## 범주값 처리
1. 일단 원-핫 인코딩 하자.
2. 원-핫 인코딩 후 입벧

In [ ]:
# AREA_ID와 DIST_CD 제거
train_df = train_df.drop(['AREA_ID', 'DIST_CD'], axis=1)

# AREA_NM에 대한 처리
top_100_area_nm = train_df.nlargest(100, 'TOTAL_ELEC')['AREA_NM'].unique()
bottom_100_area_nm = train_df.nsmallest(100, 'TOTAL_ELEC')['AREA_NM'].unique()

# DIST_NM에 대한 처리
top_100_dist_nm = train_df.nlargest(100, 'TOTAL_ELEC')['DIST_NM'].unique()
bottom_100_dist_nm = train_df.nsmallest(100, 'TOTAL_ELEC')['DIST_NM'].unique()

# 중요 범주를 표시하고 나머지는 'Other'로 묶기
train_df['AREA_NM'] = train_df['AREA_NM'].where(train_df['AREA_NM'].isin(top_100_area_nm) | train_df['AREA_NM'].isin(bottom_100_area_nm), 'Other')

train_df['DIST_NM'] = train_df['DIST_NM'].where(train_df['DIST_NM'].isin(top_100_dist_nm) | train_df['DIST_NM'].isin(bottom_100_dist_nm), 'Other')

# 원-핫 인코딩 적용 (drop_first=True는 첫 번째 카테고리를 제외)
data = pd.get_dummies(train_df, columns=['AREA_NM', 'DIST_NM'], drop_first=True)

# 결과 출력
print(data.head())

   DATA_YM  TOTAL_BIDG  FAC_NEIGH_1  FAC_NEIGH_2  FAC_CULT_MTG  FAC_RELG  \
0   202303       538.0        266.0         88.0           6.0       3.0   
1   202303       765.0         72.0        174.0           1.0       0.0   
2   202212       343.0         74.0        161.0           1.0       3.0   
3   202312       129.0          4.0         14.0           0.0       1.0   
4   202301        34.0          2.0         14.0           0.0       0.0   

   FAC_RETAIL  FAC_MEDI  FAC_YOSE  FAC_TRAIN  ...  DIST_NM_중구  \
0         2.0       3.0       0.0        0.0  ...        True   
1         2.0       0.0       1.0        0.0  ...       False   
2         0.0       2.0       6.0        0.0  ...       False   
3         0.0       0.0       0.0        0.0  ...       False   
4         1.0       0.0       1.0        0.0  ...       False   

   DIST_NM_창원시 성산구  DIST_NM_철원군  DIST_NM_청주시 서원구  DIST_NM_청주시 흥덕구  \
0            False        False            False            False   
1            F

In [ ]:
def preprocess_train_data(train_df):
    # 제거할 ID 및 DIST 목록
    train_remove_ids = [(9888, '수원시청_1'), (9946, '유천아파트앞')]
    train_remove_dist = [(42720, '홍천군'), (42730, '횡성군'), (47720, '군위군')]

    for area_id, area_name in train_remove_ids:
        train_df = train_df[~((train_df['AREA_ID'] == area_id) & (train_df['AREA_NM'] == area_name))]

    for dist_cd, dist_name in train_remove_dist:
        train_df = train_df[~((train_df['DIST_CD'] == dist_cd) & (train_df['DIST_NM'] == dist_name))]

    # 컬럼 제거
    train_df = train_df.drop(['AREA_ID', 'DIST_CD'], axis=1)

    # 중요 범주 추출
    top_100_area_nm = train_df.nlargest(100, 'TOTAL_ELEC')['AREA_NM'].unique()
    bottom_100_area_nm = train_df.nsmallest(100, 'TOTAL_ELEC')['AREA_NM'].unique()
    top_100_dist_nm = train_df.nlargest(100, 'TOTAL_ELEC')['DIST_NM'].unique()
    bottom_100_dist_nm = train_df.nsmallest(100, 'TOTAL_ELEC')['DIST_NM'].unique()

    # 범주 압축
    train_df['AREA_NM'] = train_df['AREA_NM'].where(train_df['AREA_NM'].isin(top_100_area_nm) | train_df['AREA_NM'].isin(bottom_100_area_nm), 'Other')
    train_df['DIST_NM'] = train_df['DIST_NM'].where(train_df['DIST_NM'].isin(top_100_dist_nm) | train_df['DIST_NM'].isin(bottom_100_dist_nm), 'Other')

    # NaN 처리
    num_cols = train_df.select_dtypes(include='number').columns
    cat_cols = train_df.select_dtypes(include='object').columns

    train_df[num_cols] = train_df[num_cols].fillna(train_df[num_cols].mean())
    train_df[cat_cols] = train_df[cat_cols].fillna('Other')

    # 원-핫 인코딩
    train_df_encoded = pd.get_dummies(train_df, columns=['AREA_NM', 'DIST_NM'], drop_first=True)

    # return: 전처리된 데이터, top/bottom 범주 목록 (테스트에 활용)
    return train_df_encoded, top_100_area_nm, bottom_100_area_nm, top_100_dist_nm, bottom_100_dist_nm


In [ ]:
def preprocess_test_data(test_df, top_areas, bottom_areas, top_dists, bottom_dists, reference_columns):
    # 동일한 컬럼 제거
    test_df = test_df.drop(['AREA_ID', 'DIST_CD'], axis=1)

    # 범주 압축
    test_df['AREA_NM'] = test_df['AREA_NM'].where(test_df['AREA_NM'].isin(top_areas) | test_df['AREA_NM'].isin(bottom_areas), 'Other')
    test_df['DIST_NM'] = test_df['DIST_NM'].where(test_df['DIST_NM'].isin(top_dists) | test_df['DIST_NM'].isin(bottom_dists), 'Other')

    # NaN 처리
    num_cols = test_df.select_dtypes(include='number').columns
    cat_cols = test_df.select_dtypes(include='object').columns

    test_df[num_cols] = test_df[num_cols].fillna(test_df[num_cols].mean())
    test_df[cat_cols] = test_df[cat_cols].fillna('Other')

    # 원-핫 인코딩
    test_df_encoded = pd.get_dummies(test_df, columns=['AREA_NM', 'DIST_NM'], drop_first=True)

    # 누락된 컬럼 채우고 순서 맞춤
    missing_cols = set(reference_columns) - set(test_df_encoded.columns)
    for col in missing_cols:
        test_df_encoded[col] = 0

    test_df_encoded = test_df_encoded[reference_columns]

    return test_df_encoded


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

# 시드 설정
np.random.seed(0)

# 데이터 로드
train_df = pd.read_csv('TRAIN_DATA.csv', encoding='euc-kr')
test_df = pd.read_csv('TEST_DATA.csv', encoding='euc-kr')

# 전처리 (함수 이용)
train_data_processed, top_areas, bottom_areas, top_dists, bottom_dists = preprocess_train_data(train_df)

# 타겟 변수 로그 변환
train_data_processed['TOTAL_ELEC'] = np.log1p(train_data_processed['TOTAL_ELEC'])

# 예측용 X, y
X = train_data_processed.drop(columns='TOTAL_ELEC')
y = train_data_processed['TOTAL_ELEC']

# 테스트 데이터 전처리
test_data_processed = preprocess_test_data(test_df, top_areas, bottom_areas, top_dists, bottom_dists, X.columns)

# 원-핫 인코딩을 했기 때문에 범주형 인덱스는 비워둠
cat_idxs = []
cat_dims = []

# 모델 파라미터
tabnet_params = {
    "cat_idxs": cat_idxs,
    "cat_dims": cat_dims,
    "cat_emb_dim": 1,
    "optimizer_fn": torch.optim.Adam,
    "optimizer_params": dict(lr=1e-4),
    "scheduler_params": {"step_size": 50, "gamma": 0.9},
    "scheduler_fn": torch.optim.lr_scheduler.StepLR,
    "mask_type": "entmax",
    "gamma": 1.3,
    #"loss_fn": "logcosh"
}

# 반복 및 K-Fold 설정
num_repeats = 42
num_folds = 5
rmse_scores = []
y_pred_final = []  # 최종 예측값을 저장할 리스트

for repeat in range(num_repeats):
    print(f"\n Repeat {repeat + 1}/{num_repeats}")
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=repeat)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f" ➤ Fold {fold + 1}/{num_folds}")
        X_train, X_val = X.iloc[train_idx].values, X.iloc[val_idx].values
        y_train, y_val = y.iloc[train_idx].values.reshape(-1, 1), y.iloc[val_idx].values.reshape(-1, 1)

        model = TabNetRegressor(**tabnet_params)

        model.fit(
            X_train=X_train, y_train=y_train,
            eval_set=[(X_val, y_val)],
            eval_name=['val'],
            eval_metric=['rmse'],
            max_epochs=100,
            patience=20,
            batch_size=256, virtual_batch_size=64,
            num_workers=0,
            drop_last=False
        )

        preds = model.predict(X_val).flatten()
        rmse = mean_squared_error(y_val, preds, squared=False)
        rmse_scores.append(rmse)

# 전체 반복이 끝난 후, y_pred_final에 모든 예측값이 저장됨
y_pred_final = np.concatenate(y_pred_final, axis=0)  # 모든 fold와 repeat에서 나온 예측값을 결합

print(f"\n 평균 RMSE (Across {num_repeats} repeats x {num_folds} folds): {np.mean(rmse_scores):.4f}")


🔁 Repeat 1/42
 ➤ Fold 1/5


c:\Users\sally\anaconda3\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 39.48096| val_rmse: 5.34528 |  0:00:03s
epoch 1  | loss: 38.83648| val_rmse: 8.72288 |  0:00:07s
epoch 2  | loss: 38.07483| val_rmse: 6.67571 |  0:00:11s
epoch 3  | loss: 37.39881| val_rmse: 6.92304 |  0:00:15s
epoch 4  | loss: 36.60314| val_rmse: 6.22639 |  0:00:19s
epoch 5  | loss: 35.97552| val_rmse: 6.03657 |  0:00:23s
epoch 6  | loss: 35.27765| val_rmse: 5.89713 |  0:00:27s
epoch 7  | loss: 34.47963| val_rmse: 6.13468 |  0:00:31s
epoch 8  | loss: 33.69906| val_rmse: 6.22045 |  0:00:36s
epoch 9  | loss: 32.98782| val_rmse: 6.19184 |  0:00:40s
epoch 10 | loss: 32.11109| val_rmse: 6.11273 |  0:00:43s
epoch 11 | loss: 31.3971 | val_rmse: 6.06413 |  0:00:47s
epoch 12 | loss: 30.56873| val_rmse: 6.03026 |  0:00:51s
epoch 13 | loss: 29.69859| val_rmse: 5.95094 |  0:00:55s
epoch 14 | loss: 28.72013| val_rmse: 5.8878  |  0:01:00s
epoch 15 | loss: 27.91503| val_rmse: 5.78697 |  0:01:05s
epoch 16 | loss: 26.88939| val_rmse: 5.77746 |  0:01:09s
epoch 17 | loss: 25.85254| val_

c:\Users\sally\anaconda3\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

In [24]:
print(f"\n✅ 평균 RMSE (Across {num_repeats} repeats x {num_folds} folds): {np.mean(rmse_scores):.4f}")


✅ 평균 RMSE (Across 42 repeats x 5 folds): nan


c:\Users\sally\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\sally\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


#### ✅ 상권별 예측 오차 분석 (TOP 10)

In [23]:
import matplotlib.pyplot as plt
import seaborn as sns

# 예측값 계산
y_pred_final = model.predict(X).flatten()  # 예측 결과를 y_pred_final에 저장

# 예측 오차 계산
error_df = train_df.copy()
error_df['predicted'] = y_pred_final
error_df['abs_error'] = np.abs(error_df['TOTAL_ELEC'] - error_df['predicted'])

# 상위 10개 상권 표시 (오차 기준)
top_errors = error_df.sort_values('abs_error', ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(data=top_errors, x='AREA_NM', y='abs_error', palette='Reds_r')
plt.title("상권별 예측 오차 상위 10")
plt.xlabel("상권 명")
plt.ylabel("절대 오차 (abs_error)")
plt.xticks(rotation=45)
plt.show()

KeyError: 0

#### ✅ 전체 학습 데이터로 최종 모델 재학습
- 전체 X, y로 최종 TabNet 모델 재훈련

In [ ]:
# 전체 학습 데이터로 모델 학습
final_model = TabNetRegressor(**tabnet_params)
final_model.fit(
    X_train=X.values, y_train=y.values,
    max_epochs=100,
    patience=10,
    batch_size=1024, virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

In [ ]:
# 예측 수행
y_pred = final_model.predict(test_data_processed.values).flatten()

# 결과 DataFrame 생성
submission = pd.DataFrame({'y_pred': y_pred})

# 저장
submission.to_csv("submission.csv", index=False)
print("예측 결과가 'submission.csv' 로 저장되었습니다.")